In [41]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import sys
sys.path.append('../src/')
from utils import estimate_density

In [42]:
# Embrace FIRST CLASS functions :D
def get_estimator_func(
    bw="silverman",
    grid_len=256, 
    extend=True, 
    bound_correction=False, 
    adaptive=False,
    extend_fct=0.5, 
    bw_fct=1,
    bw_return=False,
    custom_lims=None
):
    
    def func(x, bw=bw, grid_len=grid_len,
            bound_correction=bound_correction,
            adaptive=adaptive, extend_fct=extend_fct,
            bw_fct=bw_fct, bw_return=bw_return,
            custom_lims=custom_lims):
        
        return estimate_density(x=x, bw=bw, grid_len=grid_len,
                       bound_correction=bound_correction,
                       adaptive=adaptive, extend_fct=extend_fct,
                       bw_fct=bw_fct, bw_return=bw_return,
                       custom_lims=custom_lims)
        
    return func

In [93]:
def get_gmixture_rvs(size, mean, sd, wt = None):
       
    if wt is None:
        wt = np.repeat((1 / len(mean)), len(mean))
    assert len(mean) == len(sd) == len(wt)
    assert np.sum(wt) == 1
    x = np.concatenate((
        list(map(lambda m, s, w: stats.norm.rvs(m, s, int(np.round(size * w))), mean, sd, wt))
    ))
    return x

def get_gmixture_pdf(grid, mean, sd, wt=None):
    if wt is None:
        wt = np.repeat((1 / len(mean)), len(mean))
    assert len(mean) == len(sd) == len(wt)
    assert np.sum(wt) == 1
    pdf = np.average(list((map(lambda m, s: stats.norm.pdf(grid, m, s), mean, sd))), axis=0, weights=wt)
    return pdf

def get_gamma_rvs(size, shape, scale=1):
    return stats.gamma.rvs(a=shape, scale=scale, size=size)

def get_gamma_pdf(grid, shape, scale=1):
    return stats.gamma.pdf(grid, a=shape, scale=scale)

def get_logn_rvs(size, scale):
    return stats.lognorm.rvs(s=scale, size=size)

def get_logn_pdf(grid, scale):
    return stats.lognorm.pdf(grid, s=scale)

def get_beta_rvs(size, a, b):
    return stats.beta.rvs(a=b, b=b, size=size)

def get_beta_pdf(grid, a, b):
    return stats.beta.pdf(grid, a=a, b=b)

In [94]:
pdf_funcs = {
    "gaussian": get_gmixture_pdf,
    "gamma": get_gamma_pdf,
    "logn": get_logn_pdf,
    "beta": get_beta_pdf
}

rvs_funcs = {
    "gaussian": get_gmixture_rvs,
    "gamma": get_gamma_rvs,
    "logn": get_logn_rvs,
    "beta": get_beta_rvs
}

# Embrace lexical scoping bitch!
def get_pdf_func(identifier, **kwargs):
    def func(grid):
        return pdf_funcs[identifier](grid=grid, **kwargs)
    return func

def get_rvs_func(identifier, **kwargs):
    def func(size):
        return rvs_funcs[identifier](size=size, **kwargs)
    return func

In [ ]:

colums = ["iter", "pdf", "estimator", "bw", "size", "time"]
df = pd.DataFrame(columns=colums)

for i in range(niter):
    rvs = rvs_func(rvs_params, size)
    
    grid, pdf = estimator_func(rvs)
    
    squared_diff = (pdf - pdf_func(grid, pdf_params)) ** 2
    ise = trapezoid_sum(grid, squared_diff)
    
    df.loc[i] = [i + 1, pdf, estimator, bw, size, time] 